In [2]:
import pandas as pd
import glob
import os
import openai
import requests
from PIL import Image
from io import BytesIO


In [3]:
os.chdir('/Users/pranavsharma/Desktop/web-scraper/final_csv')
file_patterns = ["product_data_*_*_final.csv", "product_data_*_*_final_2.csv"]


file_list = []
for pattern in file_patterns:
    file_list.extend(glob.glob(pattern))


final_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)



In [4]:
len(final_df)

9014

In [5]:
unique_df = final_df.drop_duplicates(subset='Product Name', keep='first')

In [6]:
len(unique_df)

8888

In [7]:
os.chdir('/Users/pranavsharma/Desktop/web-scraper/final_image_urls')
file_patterns = ["image_urls_*_*_final.csv"]


file_list = []
for pattern in file_patterns:
    file_list.extend(glob.glob(pattern))


image_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)



In [8]:
unique_image_df = image_df.drop_duplicates(subset='Product Name', keep='first')

In [9]:
len(unique_image_df)

9042

In [10]:
final_df=pd.merge(unique_df, unique_image_df, on='Product Name', how='left')

In [12]:
final_df=final_df.drop(columns=['Product URL_y'])

In [13]:
final_df=final_df.rename(columns={'Product URL_x':'Product URL'})

In [14]:
filtered_df = unique_df[
    unique_df['Tags'].str.contains(r'\b(Artificial Intelligence|AI)\b', case=False, na=False) |
    unique_df['Product Description'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False) |
    unique_df['Product Meta Data'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False)
]

/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/2015976755.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df['Tags'].str.contains(r'\b(Artificial Intelligence|AI)\b', case=False, na=False) |
/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/2015976755.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_df['Product Description'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False)


In [15]:
len(filtered_df)

3915

In [ ]:
import ast
filtered_df['Product Description'].iloc[0]
image_urls=ast.literal_eval(filtered_df['Product Image URLs'][0])

In [34]:
from openai import OpenAI
client = OpenAI(base_url = "https://openai.gateway.salt-lab.org/v1/",api_key="sk-deVPbABCmY3IplUK6b22E0E6C0D94a598b58C3Af1cF3CdA5")

def analyze_products(product_info,image_urls):
  prompt=f"""

  You will be provided system description of a product, and a few images associated with the product. You need to analyze the images and see if you can fetch any additional information strictly from the images which is not specified in the product description.
  Directly state the additional information that you can fetch from the images and do not include any words which are not relevant to the images. Directly output the additional content without specifying the source of the information.
  <context>
  The following is the product description:
  {product_info}
  </contex>

  return only the additional information that you can fetch from the images in a single string in the following JSON format:
  {{"additional_info": "The additional information that you can fetch from the images."}}
    """
  
  message_content = [{"type": "text", "text": prompt}]
  for url in image_urls:
        message_content.append({
            "type": "image_url",
            "image_url": {
                "url": url,
                "detail":"low"
            }
        })
  
  final_message=[
      {
          "role":"user",
          "content":message_content
      }
  ]
  
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=final_message,
    max_tokens=300,
  )
  return response.choices[0].message.content
  
  # print(response.choices[0].message.content)

In [36]:
import ast
import json_repair
file_name='/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info_3.csv'
new_df = pd.DataFrame(columns=[
    'Product Name', 'Product Meta Data', 'Product Description', 
    'Product Vote Count', 'Bade List', 'Tags', 
    'Product URL', 'Product Image URLs', 'Additional Information'
])

cnt=0
for i,row in filtered_df.iterrows():
  cnt+=1
  if cnt<284:
    continue
  try:
    product_info=row['Product Description']
    image_urls=ast.literal_eval(row['Product Image URLs'])
    result=analyze_products(product_info,image_urls)
    res=json_repair.loads(result)
    extra_info = res.get('additional_info', '')

   
    product_df = pd.DataFrame([{
            'Product Name': row['Product Name'],
            'Product Meta Data': row['Product Meta Data'],
            'Product Description': row['Product Description'],
            'Product Vote Count': row['Product Vote Count'],
            'Bade List': row['Bade List'],
            'Tags': row['Tags'],
            'Product URL': row['Product URL'],
            'Product Image URLs': row['Product Image URLs'],
            'Additional Information': extra_info
        }])
    
    new_df = pd.concat([new_df, product_df], ignore_index=True)

    new_df.to_csv(file_name,index=False)
    print(f'Processed for {cnt}')
  except Exception as e:
    print(f"Error for product: {row['Product Name']}, exception {e}")
  # break
# product_info=final_df['Product Description'][0]
# image_urls=ast.literal_eval(final_df['Product Image URLs'][0])
# analyze_products(product_info,image_urls)

Error for product: Screenflick, exception malformed node or string: nan
Error for product: Athlabs, exception malformed node or string: nan
Error for product: Gofer, exception malformed node or string: nan


/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/2653306259.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, product_df], ignore_index=True)


Processed for 287
Processed for 288
Processed for 289
Processed for 290
Processed for 291
Processed for 292
Processed for 293
Processed for 294
Processed for 295
Processed for 296
Processed for 297
Processed for 298
Processed for 299
Processed for 300
Error for product: OFFAIGO, exception malformed node or string: nan
Error for product: NinjaRIP, exception malformed node or string: nan
Error for product: TruthPix, exception malformed node or string: nan
Error for product: Telescope, exception malformed node or string: nan
Error for product: Beeyond AI, exception malformed node or string: nan
Error for product: AnyoneCanAI, exception malformed node or string: nan
Error for product: Inrō, exception malformed node or string: nan
Error for product: Scopey, exception malformed node or string: nan
Error for product: AI Content Genie, exception malformed node or string: nan
Processed for 310
Processed for 311
Processed for 312
Processed for 313
Processed for 314
Processed for 315
Processed fo

### AI Products in the entire data (4045)

In [42]:
df1=pd.read_csv('/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info.csv')
df2=pd.read_csv('/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info_2.csv')
df3=pd.read_csv('/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info_3.csv')
df4=pd.read_csv('/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info_4_missing.csv')

final_ai_df=pd.concat([df1,df2,df3,df4], ignore_index=True)

In [43]:
print(len(final_ai_df))
# final_ai_df.to_csv('/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_extra_info.csv',index=False)

3495


In [44]:
os.chdir('/Users/pranavsharma/Desktop/web-scraper/final_csv')
file_patterns = ["product_data_*_*_final.csv", "product_data_*_*_final_2.csv"]


file_list = []
for pattern in file_patterns:
    file_list.extend(glob.glob(pattern))


final_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)
unique_df = final_df.drop_duplicates(subset='Product Name', keep='first')

filtered_df = unique_df[
    unique_df['Tags'].str.contains(r'\b(Artificial Intelligence|AI)\b', case=False, na=False) |
    unique_df['Product Description'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False) |
    unique_df['Product Meta Data'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False)
]

/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/127861792.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  unique_df['Tags'].str.contains(r'\b(Artificial Intelligence|AI)\b', case=False, na=False) |
/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/127861792.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  unique_df['Product Description'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learning)\b', case=False, na=False) |
/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/127861792.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  unique_df['Product Meta Data'].str.contains(r'\b(ai|ai\.|ml|ml\.|-ai|ai-|artificial intelligence|machine learnin

In [45]:
len(filtered_df)

4045

In [46]:
missing_products = filtered_df[~filtered_df['Product Name'].isin(final_ai_df['Product Name'])]


missing_products['Additional Information'] = ''

final_ai_df = pd.concat([final_ai_df, missing_products], ignore_index=True)

print(f"Updated final_ai_df has {len(final_ai_df)} products (including missing ones).")

output_file = '/Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_all_info.csv'  
final_ai_df = final_ai_df.drop_duplicates(subset='Product Name', keep='first')
final_ai_df.to_csv(output_file, index=False)
print(f"Updated DataFrame saved to {output_file}")


/var/folders/4g/d8pkwx5s0pjcpd5rd70lx4fc0000gn/T/ipykernel_17339/2409770577.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_products['Additional Information'] = ''


Updated final_ai_df has 4045 products (including missing ones).
Updated DataFrame saved to /Users/pranavsharma/Desktop/web-scraper/results_files/AI_products_all_info.csv


In [63]:
# image_urls

['https://ph-files.imgix.net/48f8c05d-8132-4dd6-b57d-148e4e632776.jpeg',
 'https://ph-files.imgix.net/fc954cf1-b748-4a8b-9ff6-46be05419a41.png',
 'https://ph-files.imgix.net/159041f8-ac2c-40a1-889b-d7462517a553.png',
 'https://ph-files.imgix.net/298f135c-9a87-4888-87b3-1437a97a557e.png',
 'https://ph-files.imgix.net/a9f801c0-de2c-411d-84c1-b20ea01ceb0e.png',
 'https://ph-files.imgix.net/3e0ed48b-a3af-4919-be4b-f602e5ac8a14.png',
 'https://ph-files.imgix.net/305cc1ad-6075-431d-99e2-ae536381d7b0.png',
 'https://ph-files.imgix.net/b1e9027c-1f58-4989-a28b-a91c11eecc57.png',
 'https://ph-files.imgix.net/3d565820-396c-4fef-8279-065abe9113ae.png']

In [78]:
# filtered_df['Product Description'].iloc[0]

'Discover your dream job with Jobright, your AI job search copilot. Find relevant opportunities tailored to your goals. Gain filtered matches, deep insights, and stay ahead with real-time company trends. Let Jobright find your perfect fit.'